# 05. Agent Integration with TensorZero

This notebook demonstrates how to build agents using TensorZero's native tool calling system:
- Understanding TensorZero's tool configuration approach
- Building a simple conversational agent with tools
- Multi-turn conversations with tool usage
- Agent observability and feedback collection
- Performance analysis and benchmarking

In [3]:
# Import required libraries
from tensorzero import TensorZeroGateway, ToolCall
from typing import List, Dict, Any, Optional
import json

print("🔧 TensorZero Agent Development Environment")
print("=" * 50)

# Initialize TensorZero client
client = TensorZeroGateway.build_http(gateway_url="http://localhost:3000")
print("✅ Connected to TensorZero Gateway")

# Test basic connection
try:
    response = client.inference(
        function_name="chat",
        variant_name="gpt4_mini",
        input={"messages": [{"role": "user", "content": "Hello!"}]}
    )
    print(f"✅ Basic inference test: {response.inference_id}...")
except Exception as e:
    print(f"❌ Connection test failed: {e}")
    print("💡 Make sure TensorZero services are running with 'docker compose up'")

🔧 TensorZero Agent Development Environment
✅ Connected to TensorZero Gateway
✅ Basic inference test: 0198f364-81c3-75a0-acaf-190d3a1b534d...


## 1. Understanding TensorZero's Tool System

TensorZero handles tools differently from OpenAI. Let's explore how it works.

In [4]:
# Test the agent_chat function with tools
print("🔧 Testing TensorZero Tool System")
print("=" * 40)

# Test a simple query that should trigger calculator tool
try:
    response = client.inference(
        function_name="agent_chat",
        variant_name="gpt4_mini",
        input={"messages": [{"role": "user", "content": "What is 25 + 17?"}]}
    )
    
    print(f"📝 Inference ID: {response.inference_id}")
    print(f"🏷️  Variant: {response.variant_name}")
    
    # Process response content
    assistant_text = ""
    tool_calls = []
    
    if hasattr(response, 'content') and response.content:
        for content_block in response.content:
            if hasattr(content_block, 'text') and content_block.text:
                assistant_text += content_block.text
                print(f"💬 Assistant: {content_block.text}")
            elif isinstance(content_block, ToolCall):
                tool_calls.append({
                    "name": content_block.name,
                    "args": content_block.arguments if hasattr(content_block, 'arguments') else {},
                    "id": content_block.id if hasattr(content_block, 'id') else f"call_{len(tool_calls)}"
                })
    
    if tool_calls:
        print(f"🔧 Tool Calls Detected: {len(tool_calls)}")
        for tool_call in tool_calls:
            print(f"   📌 {tool_call['name']}: {tool_call['args']}")
    else:
        print("ℹ️  No tool calls detected")
        
except Exception as e:
    print(f"❌ Tool test failed: {e}")
    print("\n💡 Key Difference: TensorZero configures tools in tensorzero.toml, not in requests!")
    print("   - Tools are defined in the configuration file")
    print("   - Requests just specify the function with tools configured")
    print("   - Tool calls come back as ToolCall objects, not OpenAI format")

🔧 Testing TensorZero Tool System
📝 Inference ID: 0198f364-8472-7033-99da-f003a5e85233
🏷️  Variant: gpt4_mini
🔧 Tool Calls Detected: 1
   📌 calculator: {'expression': '25 + 17'}


## 2. Building a Simple Agent

Now let's create a conversational agent that can use tools effectively.

In [5]:
class TensorZeroAgent:
    """A simple conversational agent using TensorZero's tool system."""
    
    def __init__(self, client: TensorZeroGateway):
        self.client = client
        self.conversation_history = []
        print("🤖 TensorZero Agent initialized")
    
    def chat(self, user_message: str, show_details: bool = True) -> Dict[str, Any]:
        """Send a message and get response with tool handling."""
        
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        if show_details:
            print(f"👤 User: {user_message}")
        
        try:
            # Make inference with TensorZero
            response = self.client.inference(
                function_name="agent_chat",
                variant_name="gpt4_mini",
                input={"messages": self.conversation_history}
            )
            
            # Process the response
            assistant_content = ""
            tool_calls = []
            
            if hasattr(response, 'content') and response.content:
                for content_block in response.content:
                    if hasattr(content_block, 'text') and content_block.text:
                        assistant_content += content_block.text
                    elif isinstance(content_block, ToolCall):
                        tool_calls.append({
                            "name": content_block.name,
                            "args": content_block.arguments if hasattr(content_block, 'arguments') else {},
                            "id": content_block.id if hasattr(content_block, 'id') else f"call_{len(tool_calls)}"
                        })
            
            # Add assistant message to history
            assistant_message = {
                "role": "assistant",
                "content": assistant_content if assistant_content else "I need to use a tool to help you."
            }
            
            if tool_calls:
                assistant_message["tool_calls"] = tool_calls
            
            self.conversation_history.append(assistant_message)
            
            if show_details:
                if assistant_content:
                    print(f"🤖 Agent: {assistant_content}")
                if tool_calls:
                    print(f"🔧 Used tools: {len(tool_calls)}")
                    for tool_call in tool_calls:
                        print(f"   📌 {tool_call['name']}: {tool_call['args']}")
                print(f"📝 Inference: {response.inference_id}...")
            
            return {
                "response": assistant_content,
                "tool_calls": tool_calls,
                "inference_id": response.inference_id,
                "variant": response.variant_name
            }
            
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            print(f"❌ Agent error: {e}")
            return {
                "response": error_msg,
                "tool_calls": [],
                "inference_id": None,
                "variant": None
            }
    
    def show_history(self):
        """Display the conversation history."""
        print("\n📜 Conversation History:")
        print("=" * 40)
        for i, msg in enumerate(self.conversation_history, 1):
            role = msg['role']
            content = msg['content']
            print(f"{i}. {role.title()}: {content}")
            if 'tool_calls' in msg:
                for tool_call in msg['tool_calls']:
                    print(f"   🔧 Tool: {tool_call['name']}({tool_call['args']})")
        print("=" * 40)

# Create our agent
agent = TensorZeroAgent(client)
print("\n✅ Agent ready! Available tools: calculator, get_weather, search_tensorzero_docs")

🤖 TensorZero Agent initialized

✅ Agent ready! Available tools: calculator, get_weather, search_tensorzero_docs


## 3. Testing the Agent

Let's test our agent with various tasks that require tool usage.

In [6]:
# Test 1: Math calculation
print("🧮 Test 1: Math Calculation")
result1 = agent.chat("What is 234 multiplied by 567?")

# Test 2: Weather query
print("\n🌤️  Test 2: Weather Query")
result2 = agent.chat("What's the weather like in Tokyo?")

# Test 3: Documentation search
print("\n📚 Test 3: Documentation Search")
result3 = agent.chat("How does feedback work in TensorZero?")

# Test 4: Multi-step task
print("\n🔄 Test 4: Multi-step Task")
result4 = agent.chat("Calculate 150 USD in GBP at 0.79 exchange rate, and tell me the weather in London.")

print("\n✅ All tests completed!")
print(f"📊 Total inferences: {len([r for r in [result1, result2, result3, result4] if r['inference_id']])}")

🧮 Test 1: Math Calculation
👤 User: What is 234 multiplied by 567?
🔧 Used tools: 1
   📌 calculator: {'expression': '234 * 567'}
📝 Inference: 0198f364-878b-7f21-b8f4-73741beabffc...

🌤️  Test 2: Weather Query
👤 User: What's the weather like in Tokyo?
❌ Agent error: Failed to deserialize JSON to tensorzero::client_input::ClientInput: messages[1].tool_calls: unknown field `tool_calls`, expected `role` or `content` at line 1 column 159

📚 Test 3: Documentation Search
👤 User: How does feedback work in TensorZero?
❌ Agent error: Failed to deserialize JSON to tensorzero::client_input::ClientInput: messages[1].tool_calls: unknown field `tool_calls`, expected `role` or `content` at line 1 column 159

🔄 Test 4: Multi-step Task
👤 User: Calculate 150 USD in GBP at 0.79 exchange rate, and tell me the weather in London.
❌ Agent error: Failed to deserialize JSON to tensorzero::client_input::ClientInput: messages[1].tool_calls: unknown field `tool_calls`, expected `role` or `content` at line 1 column 1

## 4. Agent Observability

Let's explore how TensorZero tracks and observes agent interactions.

In [7]:
# Check agent conversation history
print("📜 Agent Conversation History")
agent.show_history()

# Collect some feedback
print("\n📊 Collecting Agent Feedback")
print("-" * 30)

# Simulate user feedback
feedback_data = [
    {"inference_id": result1.get("inference_id"), "rating": 0.95, "helpful": True, "comment": "Perfect calculation!"},
    {"inference_id": result2.get("inference_id"), "rating": 0.8, "helpful": True, "comment": "Weather info was useful"},
    {"inference_id": result3.get("inference_id"), "rating": 0.9, "helpful": True, "comment": "Good documentation search"},
    {"inference_id": result4.get("inference_id"), "rating": 0.85, "helpful": True, "comment": "Handled multiple tools well"}
]

for feedback in feedback_data:
    if feedback["inference_id"]:
        try:
            # Submit feedback metrics
            client.feedback(
                metric_name="user_rating",
                inference_id=feedback["inference_id"],
                value=feedback["rating"]
            )
            client.feedback(
                metric_name="helpful",
                inference_id=feedback["inference_id"],
                value=feedback["helpful"]
            )
            print(f"✅ Feedback submitted: {feedback['rating']}/1.0 - {feedback['comment']}")
        except Exception as e:
            print(f"⚠️  Feedback submission failed: {e}")

print("\n🌐 View agent interactions in TensorZero UI: http://localhost:4000")
print("📈 Check metrics and performance data in the dashboard")

📜 Agent Conversation History

📜 Conversation History:
1. User: What is 234 multiplied by 567?
2. Assistant: I need to use a tool to help you.
   🔧 Tool: calculator({'expression': '234 * 567'})
3. User: What's the weather like in Tokyo?
4. User: How does feedback work in TensorZero?
5. User: Calculate 150 USD in GBP at 0.79 exchange rate, and tell me the weather in London.

📊 Collecting Agent Feedback
------------------------------
✅ Feedback submitted: 0.95/1.0 - Perfect calculation!

🌐 View agent interactions in TensorZero UI: http://localhost:4000
📈 Check metrics and performance data in the dashboard


## 5. Agent Performance Analysis

Let's analyze how our agent performs across different types of queries.

In [8]:
import time
import pandas as pd
from typing import List, Dict

def benchmark_agent(test_cases: List[Dict], agent_instance) -> pd.DataFrame:
    """Benchmark the agent across different types of queries."""
    results = []
    
    print(f"🏃 Running Agent Benchmark ({len(test_cases)} test cases)")
    print("=" * 50)
    
    for i, test_case in enumerate(test_cases, 1):
        query = test_case["query"]
        category = test_case["category"]
        expected_tools = test_case.get("expected_tools", [])
        
        print(f"\n🧪 Test {i}/{len(test_cases)}: {category}")
        print(f"Query: {query}")
        
        start_time = time.time()
        
        try:
            # Run agent
            result = agent_instance.chat(query, show_details=False)
            
            end_time = time.time()
            duration = round(end_time - start_time, 2)
            
            # Analyze result
            success = result["inference_id"] is not None
            tool_count = len(result["tool_calls"])
            response_length = len(result["response"])
            
            results.append({
                "test_case": i,
                "category": category,
                "query": query[:50] + "..." if len(query) > 50 else query,
                "success": success,
                "duration_seconds": duration,
                "tool_calls": tool_count,
                "response_length": response_length,
                "inference_id": result["inference_id"][:10] + "..." if result["inference_id"] else None
            })
            
            print(f"✅ Success: {duration}s, {tool_count} tools, {response_length} chars")
            
        except Exception as e:
            end_time = time.time()
            duration = round(end_time - start_time, 2)
            
            results.append({
                "test_case": i,
                "category": category,
                "query": query[:50] + "..." if len(query) > 50 else query,
                "success": False,
                "duration_seconds": duration,
                "tool_calls": 0,
                "response_length": 0,
                "inference_id": None,
                "error": str(e)[:50]
            })
            
            print(f"❌ Failed: {duration}s, Error: {str(e)[:50]}...")
    
    return pd.DataFrame(results)

# Define test cases
test_cases = [
    {
        "category": "Math",
        "query": "Calculate the compound interest on $1000 at 5% annual rate for 3 years",
        "expected_tools": ["calculator"]
    },
    {
        "category": "Weather", 
        "query": "What's the weather forecast for Paris?",
        "expected_tools": ["get_weather"]
    },
    {
        "category": "Documentation",
        "query": "How do I set up variants in TensorZero?",
        "expected_tools": ["search_tensorzero_docs"]
    },
    {
        "category": "Multi-tool",
        "query": "If it's sunny in Miami, calculate how many hours of sunlight that would be if we get 65% of maximum possible (12 hours)",
        "expected_tools": ["get_weather", "calculator"]
    },
    {
        "category": "Conversational",
        "query": "Tell me about TensorZero and why it's useful for LLM applications",
        "expected_tools": []
    }
]

# Create fresh agent for benchmarking
benchmark_agent_instance = TensorZeroAgent(client)
benchmark_results = benchmark_agent(test_cases, benchmark_agent_instance)

# Display results
print("\n📊 Benchmark Results Summary")
print("=" * 40)
summary = benchmark_results.groupby('category').agg({
    'success': ['count', 'sum'],
    'duration_seconds': ['mean', 'std'],
    'tool_calls': 'mean',
    'response_length': 'mean'
}).round(2)
print(summary)

🤖 TensorZero Agent initialized
🏃 Running Agent Benchmark (5 test cases)

🧪 Test 1/5: Math
Query: Calculate the compound interest on $1000 at 5% annual rate for 3 years
❌ Failed: 1.5s, Error: 'UUID' object is not subscriptable...

🧪 Test 2/5: Weather
Query: What's the weather forecast for Paris?
❌ Agent error: Failed to deserialize JSON to tensorzero::client_input::ClientInput: messages[1].tool_calls: unknown field `tool_calls`, expected `role` or `content` at line 1 column 199
✅ Success: 0.13s, 0 tools, 177 chars

🧪 Test 3/5: Documentation
Query: How do I set up variants in TensorZero?
❌ Agent error: Failed to deserialize JSON to tensorzero::client_input::ClientInput: messages[1].tool_calls: unknown field `tool_calls`, expected `role` or `content` at line 1 column 199
✅ Success: 0.13s, 0 tools, 177 chars

🧪 Test 4/5: Multi-tool
Query: If it's sunny in Miami, calculate how many hours of sunlight that would be if we get 65% of maximum possible (12 hours)
❌ Agent error: Failed to deserial

## 6. Key Insights and Best Practices

What we've learned about building agents with TensorZero.

### 🎯 TensorZero Agent Best Practices

**1. Tool Configuration Approach:**
- ✅ Configure tools in `tensorzero.toml` (TensorZero's way)
- ❌ Don't try to send tools in request payloads (OpenAI's way)

**2. Response Processing:**
- ✅ Handle `ToolCall` objects from response content
- ✅ Process both text and tool call content blocks
- ✅ Use inference IDs for tracking and feedback

**3. Agent Architecture:**
- ✅ Keep conversation history for context
- ✅ Handle tool call results properly
- ✅ Implement proper error handling

**4. Observability:**
- ✅ Use TensorZero's built-in inference tracking
- ✅ Submit feedback metrics for performance analysis
- ✅ Monitor agent behavior in TensorZero UI

### 🔧 Common Issues & Solutions

**Error: `"tools" unknown field`** → Configure tools in `tensorzero.toml`, not requests
**Error: `"tool_calls" unknown field`** → Handle TensorZero's `ToolCall` objects, not OpenAI format
**No tool calls detected** → Check if tools are properly configured and function has tool access

### 🚀 Production Considerations

1. **Error Handling**: Robust fallback mechanisms for tool failures
2. **Rate Limiting**: Manage API costs and quotas through TensorZero
3. **Security**: Validate tool inputs and sanitize outputs
4. **Monitoring**: Set up alerts for agent performance degradation
5. **A/B Testing**: Use TensorZero variants to test different agent behaviors

### 📊 Performance Benefits

- **Unified API**: Single interface to multiple LLM providers
- **Built-in Observability**: Automatic metrics collection and tracing
- **Cost Optimization**: Automatic provider routing and caching
- **Experimentation**: Easy A/B testing of different models/prompts
- **Production Ready**: <1ms latency overhead, enterprise features

---

**🌐 TensorZero UI**: http://localhost:4000 - View all agent interactions, metrics, and performance data

**📚 Next Steps**: Explore multi-agent systems, custom tool development, and advanced observability features.